load hf model -> test ctx + calc sliding window pplx -> linear scaling/ntk scaling -> calc pplx

In [2]:
from huggingface_hub import login, snapshot_download
login()

In [7]:
local_dir = snapshot_download("google/gemma-1.1-2b-it")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
# local_dir = '/home/zeus/.cache/huggingface/hub/models--google--gemma-1.1-2b-it/snapshots/bf4924f313df5166dee1467161e886e55f2eb4d4'
!ls -a '/home/zeus/.cache/huggingface/hub/models--google--gemma-1.1-2b-it/snapshots/bf4924f313df5166dee1467161e886e55f2eb4d4'

.		generation_config.json		  tokenizer.json
..		model-00001-of-00002.safetensors  tokenizer.model
.gitattributes	model-00002-of-00002.safetensors  tokenizer_config.json
README.md	model.safetensors.index.json
config.json	special_tokens_map.json


In [18]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from datasets import load_dataset

In [10]:
tokenizer = AutoTokenizer.from_pretrained(local_dir)

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    local_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In circuits of logic, a mind unseen,



In [19]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt").to("cuda")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [21]:
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss
        
    print(f"nll: {neg_log_likelihood}")
    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

  0%|          | 1/565 [00:01<13:28,  1.43s/it]

nll: 5.324732780456543


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.81 GiB. GPU 0 has a total capacty of 22.17 GiB of which 1.41 GiB is free. Process 53869 has 20.75 GiB memory in use. Of the allocated memory 16.71 GiB is allocated by PyTorch, and 3.82 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF